## Compare results for multiple results

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [6]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=64
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict


In [22]:
### Read validation data
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

bkgnd_dict={}
num_bkgnd=500
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=[0,1,2,3]
### Extract validation data
for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
# del samples


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy


In [23]:
df_runs=pd.DataFrame([])

In [24]:
### bkgnd: Get bkgnd chi-squares for different sigma wrt one standard sigma 
lst=[0.5,0.65,0.8,1.1]
param_label=0.65
for count,run in enumerate(lst):
    num=500
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(run)
    samples=np.load(fname,mmap_mode='r')[1000:1000+num][:,0,:,:]
    dict1={'label':'ref_%s'%(run),'fname':''}
    dict1['num_imgs']=num

    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(samples,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    
    df_runs=df_runs.append(dict1,ignore_index=True)


### Add pytorch runs

In [25]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
fldr='20210205_120608_cgan_predict_0.5_m2/'
parent_dir+=fldr
param_label=0.5
prefix='pred_%s_'%(param_label)
lst=[(9,23080),(12,29740), (15,36500), (14,34870)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    print(key)
    fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)




pred_0.5_9-23080
(1000, 128, 128)
0.9810246 -0.99782807
pred_0.5_12-29740
(1000, 128, 128)
0.98451245 -0.99669456
pred_0.5_15-36500
(1000, 128, 128)
0.99227196 -0.99521744
pred_0.5_14-34870
(1000, 128, 128)
0.9893413 -0.99713707


In [26]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
fldr='20210205_062936_cgan_predict_0.65_m2/'
parent_dir+=fldr
param_label=0.65
prefix='pred_%s_'%(param_label)
lst=[(13,32800),(11,27120),(13,31570), (14,33520), (14,34220)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    print(key)
    fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)




pred_0.65_13-32800
(1000, 128, 128)
0.9854815 -0.9973732
pred_0.65_11-27120
(1000, 128, 128)
0.9960004 -0.99870896
pred_0.65_13-31570
(1000, 128, 128)
0.98638237 -0.99762225
pred_0.65_14-33520
(1000, 128, 128)
0.986784 -0.9980204
pred_0.65_14-34220
(1000, 128, 128)
0.9844294 -0.99769014


In [27]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
fldr='20210205_122504_cgan_predict_0.8_m2/'
parent_dir+=fldr
param_label=0.8
prefix='pred_%s_'%(param_label)
lst=[(15,35620),(12,29490),(15,35860), (13,30950), (14,34380),(14,34840),(15,36320)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    print(key)
    fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


pred_0.8_15-35620
(1000, 128, 128)
0.99312824 -0.99868613
pred_0.8_12-29490
(1000, 128, 128)
0.9871935 -0.9990013
pred_0.8_15-35860
(1000, 128, 128)
0.9918073 -0.9974072
pred_0.8_13-30950
(1000, 128, 128)
0.9900308 -0.9992028
pred_0.8_14-34380
(1000, 128, 128)
0.9912386 -0.99823517
pred_0.8_14-34840
(1000, 128, 128)
0.9887223 -0.99895996
pred_0.8_15-36320
(1000, 128, 128)
0.9872924 -0.99888825


In [28]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
fldr='20210205_063709_cgan_predict_1.1_m2/'
parent_dir+=fldr
param_label=1.1
prefix='pred_%s_'%(param_label)
lst=[(9,23310),(11,27150), (11,28060), (13,31310), (13,32550), (14,35100), (15,35710)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    print(key)
    fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



pred_1.1_9-23310
(1000, 128, 128)
0.9960735 -0.9997873
pred_1.1_11-27150
(1000, 128, 128)
0.9939979 -0.9994188
pred_1.1_11-28060
(1000, 128, 128)
0.99533325 -0.9996274
pred_1.1_13-31310
(1000, 128, 128)
0.99455374 -0.9993403
pred_1.1_13-32550
(1000, 128, 128)
0.9973461 -0.9989488
pred_1.1_14-35100
(1000, 128, 128)
0.995756 -0.99956375
pred_1.1_15-35710
(1000, 128, 128)
0.9961736 -0.99918497


In [29]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_imgs', 'spec_sdev', 'spec_val'],
      dtype='object')

In [30]:
col_list=['label', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [31]:
df_runs

,label,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,ref_0.5,0.083096,0.057402,0.582520,30.617276,0.024861,0.000833,0.031615,0.691212,18.791173,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.002444865431087542, 0.0011723789932812992, ...","[0.8476326735425547, 0.5901343441767501, 0.040...","[28033.484903826506, 14915.387050003972, 10076...","[63797.578650405405, 45566.12426947682, 33708....",500.0,
1,ref_0.65,0.002137,0.000150,0.003682,21.330047,0.001197,0.000790,0.000014,1.064209,0.132164,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0027666489046338618, 0.0012731946879723012,...","[0.9984811732297435, 0.5040168644042113, 0.045...","[40941.2382371957, 20243.50045929218, 12801.71...","[79758.82236737775, 55053.35872543143, 39483.1...",500.0,
2,ref_0.8,0.092114,0.031699,0.139352,28.184271,0.053505,0.006910,0.018840,1.327151,6.040435,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0029544550115642777, 0.0013463827295575772,...","[1.1204220686248407, 0.4362629153771028, 0.045...","[42986.27588302793, 23379.908864417015, 15311....","[87694.71672732198, 61310.9330534336, 45075.16...",500.0,
3,ref_1.1,0.500300,0.159301,0.784163,32.663335,0.277254,0.063745,0.099195,1.763453,42.548654,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.003218997397904965, 0.001456866416827564, 0...","[1.2752818483821888, 0.3512199259618418, 0.043...","[57239.57037280648, 30804.390226742657, 20808....","[101921.86736687995, 73270.86380420603, 52459....",500.0,
4,pred_0.5_9-23080,0.037945,0.024661,0.347629,28.906213,0.012617,0.000667,0.010251,0.947292,27.859108,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.002295969383274363, 0.0010615015683176317, ...","[0.9352780976577847, 0.540033556784414, 0.0435...","[34450.13415493928, 20221.49803470481, 14194.5...","[68068.64840717122, 50785.48394108106, 38618.3...",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pred_0.5_12-29740,0.013461,0.011243,0.164891,24.618822,0.002174,0.000044,0.007526,0.893496,13.202897,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0023766860820345643, 0.00111135626764241, 0...","[0.9235606465319788, 0.5477769062472018, 0.042...","[29602.393560605244, 16640.32367848015, 11892....","[59524.415628517294, 44418.653905422514, 33080...",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pred_0.5_15-36500,0.030571,0.015259,0.210560,28.839380,0.015238,0.000074,0.000836,1.088861,8.079210,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.002880583824421941, 0.0013369187201759116, ...","[0.8693128360282852, 0.5751576889680026, 0.045...","[36357.244887594185, 22045.651034745915, 14080...","[68429.29694582346, 49987.74218452413, 35925.8...",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pred_0.5_14-34870,0.070350,0.045044,0.590054,29.119427,0.024605,0.000702,0.017493,1.111628,25.368209,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0028682897905562004, 0.0013286246238285741,...","[0.9618401208891401, 0.52455736010189, 0.04505...","[33910.21674698939, 20281.365319663546, 13614....","[68048.03063500246, 51571.35803494167, 38244.8...",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pred_0.65_13-32800,0.010614,0.004376,0.701063,27.303000,0.005637,0.000602,0.000221,0.732101,58.819827,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.002521931168572828, 0.0012001383587435072, ...","[0.9927102267121748, 0.509967918028207, 0.0405...","[37854.3948098571, 21968.226818976214, 14454.1...","[68963.49749234856, 50465.872188254245, 36464....",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,pred_0.65_11-27120,0.024038,0.023508,1.564756,26.903645,0.000408,0.000122,0.018216,0.941269,115.107420,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.003458569987247706, 0.0016517748109906333, ...","[1.12080466750278, 0.44042532611641216, 0.0385...","[45066.958318075594, 25141.289638564376, 15156..."

### Compare chi-squares

In [32]:
chi_keys=['label','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,ref_0.5,0.083096,0.582520,500.0,0.057402,0.024861,0.000833,0.031615,0.691212,18.791173,30.617276
1,ref_0.65,0.002137,0.003682,500.0,0.000150,0.001197,0.000790,0.000014,1.064209,0.132164,21.330047
2,ref_0.8,0.092114,0.139352,500.0,0.031699,0.053505,0.006910,0.018840,1.327151,6.040435,28.184271
3,ref_1.1,0.500300,0.784163,500.0,0.159301,0.277254,0.063745,0.099195,1.763453,42.548654,32.663335
4,pred_0.5_9-23080,0.037945,0.347629,1000.0,0.024661,0.012617,0.000667,0.010251,0.947292,27.859108,28.906213
5,pred_0.5_12-29740,0.013461,0.164891,1000.0,0.011243,0.002174,0.000044,0.007526,0.893496,13.202897,24.618822
6,pred_0.5_15-36500,0.030571,0.210560,1000.0,0.015259,0.015238,0.000074,0.000836,1.088861,8.079210,28.839380
7,pred_0.5_14-34870,0.070350,0.590054,1000.0,0.045044,0.024605,0.000702,0.017493,1.111628,25.368209,29.119427
8,pred_0.65_13-32800,0.010614,0.701063,1000.0,0.004376,0.005637,0.000602,0.000221,0.732101,58.819827,27.303000
9,pred_0.65_11-27120,0.024038,1.564756,1000.0,0.023508,0.000408,0.000122,0.018216,0.941269,115.107420,26.903645


## Plot

In [33]:
def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
#             yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            yerr2=row.spec_sdev

            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)
    
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.legend()
            plt.title("relative spectrum")
        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
#         y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']

        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        plt.xlim(0,img_size/2)


    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.title("relative spectrum")
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)    
    
    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.5, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})

    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [34]:
# Define the reference 
param_label=0.65
dict_val=bkgnd_dict[str(param_label)]

In [35]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('ref_0.5', 'ref_0.65', 'ref_0.8', 're…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [21]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,ref_0.5,0.127021,0.582520,500.0,0.070477,18.791173
1,ref_0.65,0.001601,0.003682,500.0,0.000025,0.132164
2,ref_0.8,0.089498,0.139352,500.0,0.040195,6.040435
3,ref_1.1,0.486473,0.784163,500.0,0.210177,42.548654
4,pred_0.5_9-23080,0.054434,0.347629,1000.0,0.026951,27.859108
5,pred_0.5_12-29740,0.021623,0.164891,1000.0,0.018645,13.202897
6,pred_0.5_15-36500,0.042979,0.210560,1000.0,0.004599,8.079210
7,pred_0.5_14-34870,0.097817,0.590054,1000.0,0.044267,25.368209
8,pred_0.65_13-32800,0.015316,0.701063,1000.0,0.000843,58.819827
9,pred_0.65_11-27120,0.048033,1.564756,1000.0,0.036084,115.107420


### Test 

In [21]:
df_runs.fname.values

array(['', '', '', '',
       '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_120608_cgan_predict_0.5_m2/images/inference_label-0.5_epoch-9_step-23080.npy',
       '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_120608_cgan_predict_0.5_m2/images/inference_label-0.5_epoch-12_step-29740.npy',
       '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_120608_cgan_predict_0.5_m2/images/inference_label-0.5_epoch-15_step-36500.npy',
       '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_120608_cgan_predict_0.5_m2/images/inference_label-0.5_epoch-14_step-34870.npy',
       '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_062936_cgan_predict_0.65_m2/images/inference_label-0.65_epoch-13_step-32800.npy',
       '/global/cfs/cdirs/m3363/vayy

In [24]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210112_191125_cgan_predict_1.1_m2/images/inference_spec_label-1.1_epoch-11_step-25999.npy'
# a1=np.load(fname)
# print(a1.shape)

In [1]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy'
val_arr=np.load(fname)[:2000][:,0,:,:]
print(val_arr.shape)

NameError: name 'np' is not defined

In [ ]:
f_pixel_intensity(f_invtransform(a1),normalize=True)
plt.xscale('symlog',linthreshx=50)


# img_lst=[a1]
# f_compare_pixel_intensity(img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


In [ ]:
val_arr=val_arr[:,0,:,:]

In [ ]:
img_lst=[a1,val_arr]
f_compare_spectrum(img_lst=img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


### Test histogram

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy'
print(fname)
samples=np.load(fname,mmap_mode='r')[-1000:][:,0,:,:]
print(samples.shape)